# Google OR-Tools 测试

## 测试说明

### 官方页面

https://developers.google.com/optimization/routing/cvrp

### 算法的输入
    - 两个地点之间的交通时间
    - 每个门店的需求量
    - 每辆货车的最大承载能力
### 限制条件
    - 所有门店的需求量的总和不能大于所有货车的最大承载能力总和
    - 单家门店的需求量不能大于最大单辆货车承载能力
    - 时间、需求量、承载能力均为Integer

## 运行过程

### 安装依赖包

见`requirements.txt`

### 运行程序
    - 数据存储在`./data`文件夹下
    - 时间矩阵(`distance_matrix`)使用`./gaode/distance_matrix_20221227_1657.csv`
    - 数据中假设存在1个仓库与111个门店
    - 门店需求和货车数据为随机生成, 生成代码见`./utils/data_generator.py`
    - 假设门店需求范围为 [1, 30]
    - 一共有22辆货车, 每辆车运载能力为 [25, 150]

# CVRP with simulation data

In [54]:
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
import pandas as pd
from IPython.display import display

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.genfromtxt('./data/distance_matrix.csv', delimiter=',', dtype=int)
    data['demands'] = np.genfromtxt('./data/demands.csv', delimiter=',', dtype=int)[0:112]
    data['vehicle_capacities'] = np.genfromtxt('./data/vehicle_capacities.csv', delimiter=',', dtype=int)
    data['num_vehicles'] = len(data['vehicle_capacities'])
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution and plot."""
    locations = pd.read_csv("./gaode/locations.csv")
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    trip_df = pd.DataFrame()
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(locations['name'][node_index], route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            trip_df = pd.concat([trip_df, pd.DataFrame({'vehicle_id': [vehicle_id], 'origin': [node_index], 'destination': [manager.IndexToNode(index)]})], ignore_index=True)
        plan_output += ' {0} Load({1})\n'.format(locations['name'][manager.IndexToNode(index)],
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    
    # make folium plot
    category_20 = ('#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', 
                   '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5')
    trip_df = trip_df.query('origin!=0 | destination!=0 ')
    merged_trip_locations = pd.merge(trip_df, locations[['location', 'name']], left_on='origin', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "origin_location", 'name': 'origin_name'}, inplace=True)
    merged_trip_locations = pd.merge(merged_trip_locations, locations[['location', 'name']], left_on='destination', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "destination_location", 'name': 'destination_name'}, inplace=True)
    merged_trip_locations['origin_location'] = merged_trip_locations['origin_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['destination_location'] = merged_trip_locations['destination_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['color'] = merged_trip_locations['vehicle_id'].map(dict(zip(merged_trip_locations['vehicle_id'].unique(), category_20)))

    trip_map = folium.Map([24.48, 118.08], 
                          tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7',
                          attr='高德-常规图',
                          zoom_start=13,)

    for _, row in merged_trip_locations.iterrows():
        folium.CircleMarker(row['origin_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['origin_name'], 
                           ).add_to(trip_map)
        
        folium.CircleMarker(row['destination_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['destination_name'], 
                           ).add_to(trip_map)
    
        folium.PolyLine([row['origin_location'], row['destination_location']],
                        color=row['color'], 
                       ).add_to(trip_map)
    display(trip_map)

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
   # print(solution)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 1485
Route for vehicle 0:
 厦门鹭燕制药有限公司 Load(0) ->  鹭燕大药房(同集北路店) Load(20) ->  鹭燕大药房(银莲路店) Load(24) ->  鹭燕大药房(同安分店) Load(35) ->  鹭燕大药房(大同分店) Load(38) ->  鹭燕大药房(新霞北路店) Load(46) ->  鹭燕大药房(新霞分店) Load(48) ->  鹭燕大药房(巷南分店) Load(52) ->  鹭燕大药房(马巷五星分店) Load(74) ->  燕鹭大药房(巷南路店) Load(83) ->  厦门鹭燕制药有限公司 Load(83)
Distance of the route: 141m
Load of the route: 83

Route for vehicle 1:
 厦门鹭燕制药有限公司 Load(0) ->  厦门鹭燕制药有限公司 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 厦门鹭燕制药有限公司 Load(0) ->  鹭燕大药房(马巷分店) Load(21) ->  鹭燕大药房(五权路分店) Load(22) ->  鹭燕大药房(鼓锣东六里店) Load(41) ->  鹭燕大药房(新兴分店) Load(64) ->  鹭燕大药房(新店分店) Load(72) ->  鹭燕大药房(翔安店) Load(101) ->  鹭燕大药房(城场分店) Load(110) ->  鹭燕大药房(浒井分店) Load(124) ->  厦门鹭燕制药有限公司 Load(124)
Distance of the route: 150m
Load of the route: 124

Route for vehicle 3:
 厦门鹭燕制药有限公司 Load(0) ->  鹭燕大药房(SM分店) Load(13) ->  鹭燕大药房(屿后分店) Load(21) ->  鹭燕大药房(后埔分店) Load(26) ->  鹭燕大药房(金民里分店) Load(45) ->  鹭燕大药房(祥店二路分店) Load(46) ->  鹭燕大药房(金尚分店) Load(49) ->  鹭燕大药房(

In [21]:
locations = pd.read_csv("./gaode/locations.csv")
locations

,Column1,parent,address,distance,pname,importance,biz_ext,biz_type,cityname,type,...,typecode,shopinfo,poiweight,childtype,adname,name,location,tel,shopid,id
0,111,[],灌口镇灌南工业区安仁大道1599号,[],福建省,[],[],[],厦门市,公司企业;公司;医药公司,...,170203,0,[],[],集美区,厦门鹭燕制药有限公司,"118.007317,24.598948",0592-8129302,[],B025004PHK
1,0,[],天英二里26、27号,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],集美区,鹭燕大药房(浒井分店),"118.101334,24.604923",0592-6036378,[],B0FFFWA0TG
2,1,B025003MZA,SM城市广场负一层056号,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],201,湖里区,鹭燕大药房(SM二分店),"118.127344,24.500222",0592-5597622,[],B0FFF3DASO
3,2,[],诚毅北大街61号110,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],集美区,鹭燕大药房(软三分店),"118.044874,24.614532",0592-6683686,[],B0FFKPZJLA
4,3,[],岑西路125号202店面,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],集美区,鹭燕大药房(集美岑西分店),"118.095742,24.573115",0592-5731139,[],B0FFFS6BYL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,106,[],马巷镇巷南路54号随缘百货对面,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],翔安区,燕鹭大药房(巷南路店),"118.253242,24.662151",[],[],B0G0ASU1J6
108,107,[],马巷镇民安路60号,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],翔安区,鹭燕大药房(马巷分店),"118.250164,24.659526",0592-7620280,[],B02500TG8Y
109,108,B02500QFHK,大同路288-290号,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],202,思明区,鹭燕大药房(思北大同分店),"118.080348,24.456965",0592-2100551,[],B02500S7YU
110,109,[],祥平西路223、225、227号,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],同安区,鹭燕大药房(同安分店),"118.147349,24.727507",0592-7360028,[],B02500QKZK


# CVRP with 7.21 distance matrix scraped from Gaode


In [41]:
# Simulated demands and vehicle

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.genfromtxt('./data/dist_mat.csv', delimiter=',', dtype=int)
    data['demands'] = np.genfromtxt('./data/demands.csv', delimiter=',', dtype=int)
    data['vehicle_capacities'] = np.genfromtxt('./data/vehicle_capacities.csv', delimiter=',', dtype=int)
    data['num_vehicles'] = len(data['vehicle_capacities'])
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution and plot."""
    locations = pd.read_csv('/Users/wangshuo/Downloads/PyVRP/2023-07-21_Xiamen.csv')
    locations['Indices'] = locations.index
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    trip_df = pd.DataFrame()
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(locations['Indices'][node_index], route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            trip_df = pd.concat([trip_df, pd.DataFrame({'vehicle_id': [vehicle_id], 'origin': [node_index], 'destination': [manager.IndexToNode(index)]})], ignore_index=True)
        plan_output += ' {0} Load({1})\n'.format(locations['Indices'][manager.IndexToNode(index)],
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    
    # make folium plot
    category_20 = ('#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', 
                   '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5')
    trip_df = trip_df.query('origin!=0 | destination!=0 ')
    merged_trip_locations = pd.merge(trip_df, locations[['location', 'Indices']], left_on='origin', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "origin_location", 'Indices': 'origin_name'}, inplace=True)
    merged_trip_locations = pd.merge(merged_trip_locations, locations[['location', 'Indices']], left_on='destination', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "destination_location", 'Indices': 'destination_name'}, inplace=True)
    merged_trip_locations['origin_location'] = merged_trip_locations['origin_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['destination_location'] = merged_trip_locations['destination_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['color'] = merged_trip_locations['vehicle_id'].map(dict(zip(merged_trip_locations['vehicle_id'].unique(), category_20)))

    trip_map = folium.Map([24.48, 118.08], 
                          tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7',
                          attr='高德-常规图',
                          zoom_start=13,)

    for _, row in merged_trip_locations.iterrows():
        folium.CircleMarker(row['origin_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['origin_name'], 
                           ).add_to(trip_map)
        
        folium.CircleMarker(row['destination_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['destination_name'], 
                           ).add_to(trip_map)
    
        folium.PolyLine([row['origin_location'], row['destination_location']],
                        color=row['color'], 
                       ).add_to(trip_map)
    display(trip_map)

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
   # print(solution)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 1317188
Route for vehicle 0:
 0 Load(0) ->  210 Load(5) ->  333 Load(29) ->  304 Load(47) ->  163 Load(71) ->  87 Load(86) ->  99 Load(112) ->  43 Load(122) ->  328 Load(134) ->  7 Load(145) ->  161 Load(155) ->  108 Load(184) ->  110 Load(211) ->  218 Load(225) ->  100 Load(242) ->  281 Load(257) ->  284 Load(264) ->  196 Load(268) ->  86 Load(290) ->  68 Load(317) ->  254 Load(342) ->  52 Load(354) ->  50 Load(380) ->  295 Load(400) ->  287 Load(402) ->  314 Load(409) ->  115 Load(431) ->  279 Load(434) ->  97 Load(461) ->  149 Load(484) ->  141 Load(504) ->  56 Load(525) ->  137 Load(546) ->  223 Load(554) ->  24 Load(570) ->  156 Load(575) ->  102 Load(591) ->  159 Load(615) ->  22 Load(630) ->  23 Load(645) ->  177 Load(658) ->  142 Load(675) ->  113 Load(688) ->  341 Load(713) ->  157 Load(721) ->  27 Load(739) ->  33 Load(749) ->  32 Load(758) ->  31 Load(772) ->  30 Load(795) ->  29 Load(823) ->  28 Load(846) ->  92 Load(869) ->  140 Load(897) ->  319 Load(902) ->  3

In [36]:
# Simulated demands
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.genfromtxt('./data/dist_mat.csv', delimiter=',', dtype=int)
    data['demands'] = np.genfromtxt('./data/demands.csv', delimiter=',', dtype=int)
    data['vehicle_capacities'] = np.genfromtxt('./data/vehicle.csv', delimiter=',', dtype=int)
    data['num_vehicles'] = len(data['vehicle_capacities'])
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution and plot."""
    locations = pd.read_csv('/Users/wangshuo/Downloads/PyVRP/2023-07-21_Xiamen.csv')
    locations['Indices'] = locations.index
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    trip_df = pd.DataFrame()
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(locations['Indices'][node_index], route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            trip_df = pd.concat([trip_df, pd.DataFrame({'vehicle_id': [vehicle_id], 'origin': [node_index], 'destination': [manager.IndexToNode(index)]})], ignore_index=True)
        plan_output += ' {0} Load({1})\n'.format(locations['Indices'][manager.IndexToNode(index)],
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    
    # make folium plot
    category_20 = ('#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', 
                   '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5')
    trip_df = trip_df.query('origin!=0 | destination!=0 ')
    merged_trip_locations = pd.merge(trip_df, locations[['location', 'Indices']], left_on='origin', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "origin_location", 'Indices': 'origin_name'}, inplace=True)
    merged_trip_locations = pd.merge(merged_trip_locations, locations[['location', 'Indices']], left_on='destination', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "destination_location", 'Indices': 'destination_name'}, inplace=True)
    merged_trip_locations['origin_location'] = merged_trip_locations['origin_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['destination_location'] = merged_trip_locations['destination_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['color'] = merged_trip_locations['vehicle_id'].map(dict(zip(merged_trip_locations['vehicle_id'].unique(), category_20)))

    trip_map = folium.Map([24.48, 118.08], 
                          tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7',
                          attr='高德-常规图',
                          zoom_start=13,)

    for _, row in merged_trip_locations.iterrows():
        folium.CircleMarker(row['origin_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['origin_name'], 
                           ).add_to(trip_map)
        
        folium.CircleMarker(row['destination_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['destination_name'], 
                           ).add_to(trip_map)
    
        folium.PolyLine([row['origin_location'], row['destination_location']],
                        color=row['color'], 
                       ).add_to(trip_map)
    display(trip_map)

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
   # print(solution)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 675393
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 3:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 4:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 5:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 6:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 7:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 8:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 9:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 10:
 0 Load(0) ->  0 Loa

In [44]:
#All real data
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.genfromtxt('./data/dist_mat.csv', delimiter=',', dtype=int)
    data['demands'] = np.genfromtxt('./data/demand.csv', delimiter=',', dtype=int)
    data['vehicle_capacities'] = np.genfromtxt('./data/vehicle.csv', delimiter=',', dtype=int)
    data['num_vehicles'] = len(data['vehicle_capacities'])
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution and plot."""
    locations = pd.read_csv('/Users/wangshuo/Downloads/PyVRP/2023-07-21_Xiamen.csv')
    locations['Indices'] = locations.index
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    trip_df = pd.DataFrame()
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(locations['Indices'][node_index], route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            trip_df = pd.concat([trip_df, pd.DataFrame({'vehicle_id': [vehicle_id], 'origin': [node_index], 'destination': [manager.IndexToNode(index)]})], ignore_index=True)
        plan_output += ' {0} Load({1})\n'.format(locations['Indices'][manager.IndexToNode(index)],
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    
    # make folium plot
    category_20 = ('#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', 
                   '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5')
    trip_df = trip_df.query('origin!=0 | destination!=0 ')
    merged_trip_locations = pd.merge(trip_df, locations[['location', 'Indices']], left_on='origin', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "origin_location", 'Indices': 'origin_name'}, inplace=True)
    merged_trip_locations = pd.merge(merged_trip_locations, locations[['location', 'Indices']], left_on='destination', right_index=True, how='left')
    merged_trip_locations.rename(columns={'location': "destination_location", 'Indices': 'destination_name'}, inplace=True)
    merged_trip_locations['origin_location'] = merged_trip_locations['origin_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['destination_location'] = merged_trip_locations['destination_location'].apply(lambda x: [float(i) for i in x.split(",")][::-1])
    merged_trip_locations['color'] = merged_trip_locations['vehicle_id'].map(dict(zip(merged_trip_locations['vehicle_id'].unique(), category_20)))

    trip_map = folium.Map([24.48, 118.08], 
                          tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7',
                          attr='高德-常规图',
                          zoom_start=13,)

    for _, row in merged_trip_locations.iterrows():
        folium.CircleMarker(row['origin_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['origin_name'], 
                           ).add_to(trip_map)
        
        folium.CircleMarker(row['destination_location'],
                            radius=3,
                            fill_color=row['color'], 
                            color=row['color'], 
                            popup=row['destination_name'], 
                           ).add_to(trip_map)
    
        folium.PolyLine([row['origin_location'], row['destination_location']],
                        color=row['color'], 
                       ).add_to(trip_map)
    display(trip_map)

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
   # print(solution)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()